In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline

In [80]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import pandas as pd
import featuretools as ft
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
import datetime
from datetime import date
from pathlib import Path

In [3]:
# to make this notebook's output stable across runs
np.random.seed(42)

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

from IPython.display import display
pd.options.display.max_columns = 50
pd.options.display.html.table_schema = True

## Functions

In [4]:
def add_datepart(df, fldname, drop=True, time=False):
    """Helper function that adds columns relevant to a date."""
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

## Constants

In [5]:
TOTAL_NUMBERS = 10000

## Load Data

In [6]:
PATH = Path("datasets/lotto")
DATASET = PATH/'data_all.csv'

In [7]:
# df will store the original dataset
dataset = pd.read_csv(DATASET, parse_dates=['DrawDate'], dtype={'PrizeType': str})
dataset.columns

Index(['DrawNo', 'DrawDate', 'PrizeType', 'LuckyNo'], dtype='object')

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104673 entries, 0 to 104672
Data columns (total 4 columns):
DrawNo       104673 non-null int64
DrawDate     104673 non-null datetime64[ns]
PrizeType    104673 non-null object
LuckyNo      104673 non-null int64
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 3.2+ MB


In [9]:
dataset.dtypes

DrawNo                int64
DrawDate     datetime64[ns]
PrizeType            object
LuckyNo               int64
dtype: object

In [10]:
dataset.nunique()

DrawNo        4551
DrawDate      4551
PrizeType       23
LuckyNo      10000
dtype: int64

## Train Test Split

In [11]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(dataset, test_size=0.20, shuffle=False)
combined_df = [train_df, test_df]

display(len(train_df))
display(len(test_df))

83738

20935

## Feature generation

#### Days since last exact match

In [12]:
df = dataset.copy()

In [13]:
df['DaysSinceLastExactMatch'] = 0

result = df.loc[df.LuckyNo == 1234]
display(len(result), result)

10

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch
231,41792,1992-05-28,2ndPrizeNo,1234,0
18696,121997,1997-07-12,SpecialNo7,1234,0
35191,193801,2001-12-26,2ndPrizeNo,1234,0
47146,245705,2005-02-13,SpecialNo6,1234,0
47753,248405,2005-04-06,ConsolationNo2,1234,0
49421,255605,2005-09-11,SpecialNo4,1234,0
71468,351511,2011-04-24,ConsolationNo4,1234,0
71528,351711,2011-04-27,SpecialNo8,1234,0
98841,470517,2017-12-27,ConsolationNo7,1234,0
101068,480218,2018-07-15,ConsolationNo3,1234,0


In [14]:
datetime.datetime.strptime('1997-07-12', '%Y-%m-%d').date() - datetime.datetime.strptime('1992-05-28', '%Y-%m-%d').date()

datetime.timedelta(days=1871)

In [15]:
# df_temp['DaysSinceLastExactMatch'] = (df_temp.DrawDate - df_temp.loc[df_temp.LuckyNo == 9999].DrawDate.shift(1)).dt.days
# df_temp.loc[df_temp.LuckyNo == 9999].head(3)
matched_dates = df.loc[df.LuckyNo == 1234].DrawDate.values
days = np.diff(matched_dates).astype('timedelta64[D]')
days = np.insert(days, 0,0)
display(matched_dates, days)
len(days)

array(['1992-05-28T00:00:00.000000000', '1997-07-12T00:00:00.000000000',
       '2001-12-26T00:00:00.000000000', '2005-02-13T00:00:00.000000000',
       '2005-04-06T00:00:00.000000000', '2005-09-11T00:00:00.000000000',
       '2011-04-24T00:00:00.000000000', '2011-04-27T00:00:00.000000000',
       '2017-12-27T00:00:00.000000000', '2018-07-15T00:00:00.000000000'],
      dtype='datetime64[ns]')

array([   0, 1871, 1628, 1145,   52,  158, 2051,    3, 2436,  200],
      dtype='timedelta64[D]')

10

In [16]:
def get_elapsed_days(data, no, fld, func):
    """Get elapsed between the draw dates"""
    df_filtered = data[func(data.LuckyNo, no)]
    days = np.absolute(np.diff(df_filtered.DrawDate.values).astype('timedelta64[D]')).astype('int64')
    days = np.insert(days, 0,0)
    for idx, val in zip(df_filtered.index, days):
        data.at[idx, fld] = val

In [17]:
def exact_match(lucky_no, no):
    return lucky_no == no

In [18]:
df.sort_values(by=['DrawDate'], ascending=True, inplace=True)

In [19]:
for no in range(0, TOTAL_NUMBERS):
    if no % 1000 == 0:
        print('Processing %s of 10000' % str(no))
    get_elapsed_days(df, no, 'DaysSinceLastExactMatch', exact_match)

Processing 0 of 10000
Processing 1000 of 10000
Processing 2000 of 10000
Processing 3000 of 10000
Processing 4000 of 10000
Processing 5000 of 10000
Processing 6000 of 10000
Processing 7000 of 10000
Processing 8000 of 10000
Processing 9000 of 10000


In [20]:
display(df[df.LuckyNo==1234])

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch
231,41792,1992-05-28,2ndPrizeNo,1234,0
18696,121997,1997-07-12,SpecialNo7,1234,1871
35191,193801,2001-12-26,2ndPrizeNo,1234,1628
47146,245705,2005-02-13,SpecialNo6,1234,1145
47753,248405,2005-04-06,ConsolationNo2,1234,52
49421,255605,2005-09-11,SpecialNo4,1234,158
71468,351511,2011-04-24,ConsolationNo4,1234,2051
71528,351711,2011-04-27,SpecialNo8,1234,3
98841,470517,2017-12-27,ConsolationNo7,1234,2436
101068,480218,2018-07-15,ConsolationNo3,1234,200


#### Days Until Next Exact Match

In [21]:
df['DaysUntilNextExactMatch'] = 0
df.tail(20)

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch
104670,495819,2019-06-09,SpecialNo7,5050,120,0
104667,495819,2019-06-09,SpecialNo4,3669,14,0
104668,495819,2019-06-09,SpecialNo5,9517,603,0
104669,495819,2019-06-09,SpecialNo6,1844,1187,0
104662,495819,2019-06-09,ConsolationNo9,3596,2954,0
104666,495819,2019-06-09,SpecialNo3,1218,1628,0
104661,495819,2019-06-09,ConsolationNo8,6788,407,0
104655,495819,2019-06-09,ConsolationNo2,4622,176,0
104659,495819,2019-06-09,ConsolationNo6,3153,449,0
104658,495819,2019-06-09,ConsolationNo5,1132,189,0


In [22]:
df.sort_values(by=['DrawDate'], ascending=False, inplace=True)

In [23]:
for no in range(0, TOTAL_NUMBERS):
    if no % 1000 == 0:
        print('Processing %s of 10000' % str(no))
    get_elapsed_days(df, no, 'DaysUntilNextExactMatch', exact_match)

Processing 0 of 10000
Processing 1000 of 10000
Processing 2000 of 10000
Processing 3000 of 10000
Processing 4000 of 10000
Processing 5000 of 10000
Processing 6000 of 10000
Processing 7000 of 10000
Processing 8000 of 10000
Processing 9000 of 10000


In [24]:
display(df[df.LuckyNo==1234])

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch
101068,480218,2018-07-15,ConsolationNo3,1234,200,0
98841,470517,2017-12-27,ConsolationNo7,1234,2436,200
71528,351711,2011-04-27,SpecialNo8,1234,3,2436
71468,351511,2011-04-24,ConsolationNo4,1234,2051,3
49421,255605,2005-09-11,SpecialNo4,1234,158,2051
47753,248405,2005-04-06,ConsolationNo2,1234,52,158
47146,245705,2005-02-13,SpecialNo6,1234,1145,52
35191,193801,2001-12-26,2ndPrizeNo,1234,1628,1145
18696,121997,1997-07-12,SpecialNo7,1234,1871,1628
231,41792,1992-05-28,2ndPrizeNo,1234,0,1871


#### Days Since Last Any Match

In [25]:
def pad(val):
    return str(val).zfill(4)

In [26]:
from itertools import permutations

def get_permutations(no):
    no_list = []
    for p in list(set(permutations(pad(no)))):
        no_list.append(int(''.join(p)))
    return no_list
    
def any_match(lucky_no, no):
    return lucky_no.isin(get_permutations(no))

In [27]:
df['DaysSinceLastAnyMatch'] = 0
df.sort_values(by=['DrawDate'], ascending=True, inplace=True)
display(df[df.LuckyNo==1234])

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch
231,41792,1992-05-28,2ndPrizeNo,1234,0,1871,0
18696,121997,1997-07-12,SpecialNo7,1234,1871,1628,0
35191,193801,2001-12-26,2ndPrizeNo,1234,1628,1145,0
47146,245705,2005-02-13,SpecialNo6,1234,1145,52,0
47753,248405,2005-04-06,ConsolationNo2,1234,52,158,0
49421,255605,2005-09-11,SpecialNo4,1234,158,2051,0
71468,351511,2011-04-24,ConsolationNo4,1234,2051,3,0
71528,351711,2011-04-27,SpecialNo8,1234,3,2436,0
98841,470517,2017-12-27,ConsolationNo7,1234,2436,200,0
101068,480218,2018-07-15,ConsolationNo3,1234,200,0,0


In [28]:
for no in range(0, TOTAL_NUMBERS):
    if no % 1000 == 0:
        print('Processing %s of 10000' % str(no))
    get_elapsed_days(df, no, 'DaysSinceLastAnyMatch', any_match)

# get_elapsed_days(df, 1234, 'DaysSinceLastAnyMatch', any_match)

display(df[df.LuckyNo.isin(get_permutations(123))].head(10))

Processing 0 of 10000
Processing 1000 of 10000
Processing 2000 of 10000
Processing 3000 of 10000
Processing 4000 of 10000
Processing 5000 of 10000
Processing 6000 of 10000
Processing 7000 of 10000
Processing 8000 of 10000
Processing 9000 of 10000


,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch
294,41992,1992-06-03,SpecialNo5,213,0,15,0
458,42692,1992-06-18,SpecialNo8,213,15,42,15
590,43292,1992-07-02,SpecialNo2,2310,0,462,14
857,44492,1992-07-30,ConsolationNo3,213,42,356,28
2042,49592,1992-11-26,SpecialNo5,3201,0,143,119
2344,50892,1992-12-27,SpecialNo8,1230,0,500,31
2633,52193,1993-01-27,ConsolationNo8,1302,0,2771,31
2869,53193,1993-02-18,SpecialNo4,231,0,307,22
3444,55693,1993-04-18,SpecialNo4,3201,143,812,59
3728,56993,1993-05-19,3rdPrizeNo,1023,0,28,31


#### Days Until Next Any Match

In [29]:
df['DaysUntilNextAnyMatch'] = 0
df.sort_values(by=['DrawDate'], ascending=False, inplace=True)
display(df[df.LuckyNo.isin(get_permutations(123))].head(10))

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch
104291,494219,2019-05-04,ConsolationNo6,1032,186,0,13,0
104187,493719,2019-04-21,SpecialNo7,312,602,0,11,0
104054,493219,2019-04-10,3rdPrizeNo,123,973,0,115,0
102819,487818,2018-12-16,ConsolationNo6,3201,686,0,47,0
102270,485418,2018-10-30,ConsolationNo9,1032,254,186,97,0
101163,480618,2018-07-25,ConsolationNo6,213,1246,0,49,0
100638,478318,2018-06-06,SpecialNo1,3120,2863,0,81,0
99784,474618,2018-03-17,ConsolationNo7,2301,52,0,27,0
99461,473218,2018-02-18,ConsolationNo6,1032,623,254,25,0
99143,471818,2018-01-24,SpecialNo1,2301,672,52,98,0


In [30]:
for no in range(0, TOTAL_NUMBERS):
    if no % 1000 == 0:
        print('Processing %s of 10000' % str(no))
    get_elapsed_days(df, no, 'DaysUntilNextAnyMatch', any_match)

display(df[df.LuckyNo.isin(get_permutations(123))].head(10))

Processing 0 of 10000
Processing 1000 of 10000
Processing 2000 of 10000
Processing 3000 of 10000
Processing 4000 of 10000
Processing 5000 of 10000
Processing 6000 of 10000
Processing 7000 of 10000
Processing 8000 of 10000
Processing 9000 of 10000


,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch
104291,494219,2019-05-04,ConsolationNo6,1032,186,0,13,0
104187,493719,2019-04-21,SpecialNo7,312,602,0,11,13
104054,493219,2019-04-10,3rdPrizeNo,123,973,0,115,11
102819,487818,2018-12-16,ConsolationNo6,3201,686,0,47,115
102270,485418,2018-10-30,ConsolationNo9,1032,254,186,97,47
101163,480618,2018-07-25,ConsolationNo6,213,1246,0,49,97
100638,478318,2018-06-06,SpecialNo1,3120,2863,0,81,49
99784,474618,2018-03-17,ConsolationNo7,2301,52,0,27,81
99461,473218,2018-02-18,ConsolationNo6,1032,623,254,25,27
99143,471818,2018-01-24,SpecialNo1,2301,672,52,98,25


#### Extract digits by position

In [65]:
df['LuckyNo_str'] = df['LuckyNo'].apply(str).apply(pad)
df['1st_digit'] = df['LuckyNo_str'].str[0:1].apply(int)
df['2nd_digit'] = df['LuckyNo_str'].str[1:2].apply(int)
df['3rd_digit'] = df['LuckyNo_str'].str[2:3].apply(int)
df['4th_digit'] = df['LuckyNo_str'].str[3:4].apply(int)
df.drop(columns=['LuckyNo_str'], axis=1, inplace=True)
display(df[df.LuckyNo.isin(get_permutations(123))].head(10))

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,1st_digit,2st_digit,3st_digit,4st_digit,DrawYear,DrawMonth,DrawWeek,DrawDay,DrawDayofweek,DrawDayofyear,CumProbaExactMatch,CumProbaAnyMatch,2nd_digit,3rd_digit,4th_digit
294,41992,1992-06-03,SpecialNo5,213,0,15,0,15,0,2,1,3,1992,6,23,3,2,155,0.01,0.01,2,1,3
458,42692,1992-06-18,SpecialNo8,213,15,42,15,14,0,2,1,3,1992,6,25,18,3,170,0.02,0.02,2,1,3
590,43292,1992-07-02,SpecialNo2,2310,0,462,14,28,2,3,1,0,1992,7,27,2,3,184,0.01,0.03,3,1,0
857,44492,1992-07-30,ConsolationNo3,213,42,356,28,119,0,2,1,3,1992,7,31,30,3,212,0.03,0.04,2,1,3
2042,49592,1992-11-26,SpecialNo5,3201,0,143,119,31,3,2,0,1,1992,11,48,26,3,331,0.01,0.05,2,0,1
2344,50892,1992-12-27,SpecialNo8,1230,0,500,31,31,1,2,3,0,1992,12,52,27,6,362,0.01,0.06,2,3,0
2633,52193,1993-01-27,ConsolationNo8,1302,0,2771,31,22,1,3,0,2,1993,1,4,27,2,27,0.01,0.07,3,0,2
2869,53193,1993-02-18,SpecialNo4,231,0,307,22,59,0,2,3,1,1993,2,7,18,3,49,0.01,0.08,2,3,1
3444,55693,1993-04-18,SpecialNo4,3201,143,812,59,31,3,2,0,1,1993,4,15,18,6,108,0.02,0.09,2,0,1
3728,56993,1993-05-19,3rdPrizeNo,1023,0,28,31,28,1,0,2,3,1993,5,20,19,2,139,0.01,0.10,0,2,3


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104673 entries, 0 to 104672
Data columns (total 20 columns):
DrawNo                     104673 non-null int64
DrawDate                   104673 non-null datetime64[ns]
PrizeType                  104673 non-null object
LuckyNo                    104673 non-null int64
DaysSinceLastExactMatch    104673 non-null int64
DaysUntilNextExactMatch    104673 non-null int64
DaysSinceLastAnyMatch      104673 non-null int64
DaysUntilNextAnyMatch      104673 non-null int64
1st_digit                  104673 non-null int64
2st_digit                  104673 non-null int64
3st_digit                  104673 non-null int64
4st_digit                  104673 non-null int64
DrawYear                   104673 non-null int64
DrawMonth                  104673 non-null int64
DrawWeek                   104673 non-null int64
DrawDay                    104673 non-null int64
DrawDayofweek              104673 non-null int64
DrawDayofyear              104673 non-null int

#### Add Date Parts

In [32]:
columns_to_drop = ['DrawIs_month_end','DrawIs_month_start', 'DrawIs_quarter_end', 'DrawIs_quarter_start', 'DrawIs_year_end', 'DrawIs_year_start' ,'DrawElapsed']
add_datepart(df, 'DrawDate', drop=False)
df.drop(columns=columns_to_drop, axis=1, inplace=True)
display(df[df.LuckyNo.isin(get_permutations(123))].head(10))

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,1st_digit,2st_digit,3st_digit,4st_digit,DrawYear,DrawMonth,DrawWeek,DrawDay,DrawDayofweek,DrawDayofyear
104291,494219,2019-05-04,ConsolationNo6,1032,186,0,13,0,1,0,3,2,2019,5,18,4,5,124
104187,493719,2019-04-21,SpecialNo7,312,602,0,11,13,0,3,1,2,2019,4,16,21,6,111
104054,493219,2019-04-10,3rdPrizeNo,123,973,0,115,11,0,1,2,3,2019,4,15,10,2,100
102819,487818,2018-12-16,ConsolationNo6,3201,686,0,47,115,3,2,0,1,2018,12,50,16,6,350
102270,485418,2018-10-30,ConsolationNo9,1032,254,186,97,47,1,0,3,2,2018,10,44,30,1,303
101163,480618,2018-07-25,ConsolationNo6,213,1246,0,49,97,0,2,1,3,2018,7,30,25,2,206
100638,478318,2018-06-06,SpecialNo1,3120,2863,0,81,49,3,1,2,0,2018,6,23,6,2,157
99784,474618,2018-03-17,ConsolationNo7,2301,52,0,27,81,2,3,0,1,2018,3,11,17,5,76
99461,473218,2018-02-18,ConsolationNo6,1032,623,254,25,27,1,0,3,2,2018,2,7,18,6,49
99143,471818,2018-01-24,SpecialNo1,2301,672,52,98,25,2,3,0,1,2018,1,4,24,2,24


#### Calculate Combinations Per X Draws

In [33]:
df.sort_values(by=['DrawDate', 'PrizeType'], ascending=False, inplace=True)
df.head(23)

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,1st_digit,2st_digit,3st_digit,4st_digit,DrawYear,DrawMonth,DrawWeek,DrawDay,DrawDayofweek,DrawDayofyear
104672,495819,2019-06-09,SpecialNo9,2976,2066,0,39,0,2,9,7,6,2019,6,23,9,6,160
104671,495819,2019-06-09,SpecialNo8,9073,592,0,0,0,9,0,7,3,2019,6,23,9,6,160
104670,495819,2019-06-09,SpecialNo7,5050,120,0,120,0,5,0,5,0,2019,6,23,9,6,160
104669,495819,2019-06-09,SpecialNo6,1844,1187,0,15,0,1,8,4,4,2019,6,23,9,6,160
104668,495819,2019-06-09,SpecialNo5,9517,603,0,7,0,9,5,1,7,2019,6,23,9,6,160
104667,495819,2019-06-09,SpecialNo4,3669,14,0,14,0,3,6,6,9,2019,6,23,9,6,160
104666,495819,2019-06-09,SpecialNo3,1218,1628,0,147,0,1,2,1,8,2019,6,23,9,6,160
104665,495819,2019-06-09,SpecialNo2,791,613,0,28,0,0,7,9,1,2019,6,23,9,6,160
104664,495819,2019-06-09,SpecialNo10,6223,735,0,25,0,6,2,2,3,2019,6,23,9,6,160
104663,495819,2019-06-09,SpecialNo1,9569,705,0,95,0,9,5,6,9,2019,6,23,9,6,160


In [34]:
# Derive number of combinations for the past X draw
X = 3
total_numbers = 23 * X
print(total_numbers)

69


In [35]:
def get_combinations(df, dates):
    total_distinct_combinations = 0
    total_random_combinations = 0
    combination_list = {}
    
    for no in df[df.DrawDate.isin(dates)].LuckyNo:
        found = False
        for k, v in combination_list.items():
            perms = get_permutations(k)
            if no in perms:
                found = True
                combination_list[k] = combination_list.get(k) + 1
            
        if not found:
            combination_list[no] = 1
    return combination_list


# get_combinations(df,['2019-06-09', '2019-06-08']) 
# Past 10 draws
result = get_combinations(df,df.DrawDate[0:total_numbers].unique()) 
print(total_numbers, len(result), result)

69 64 {2976: 1, 9073: 3, 5050: 1, 1844: 1, 9517: 1, 3669: 1, 1218: 1, 791: 1, 6223: 1, 9569: 1, 3596: 1, 6788: 1, 8902: 1, 3153: 1, 1132: 1, 1808: 1, 483: 1, 4622: 1, 5031: 1, 8611: 1, 1219: 1, 1232: 1, 6775: 1, 7351: 1, 1144: 1, 4703: 1, 4206: 1, 3925: 1, 7287: 1, 5228: 1, 2927: 1, 4031: 1, 9531: 2, 6964: 1, 4591: 2, 6: 1, 6801: 1, 6894: 1, 7829: 1, 3397: 1, 6780: 2, 7758: 1, 2971: 1, 307: 1, 1715: 1, 4819: 1, 7001: 1, 1314: 1, 3252: 1, 6090: 1, 1826: 1, 2177: 1, 1678: 1, 9927: 1, 977: 1, 9713: 1, 5901: 1, 1904: 1, 4486: 1, 6276: 1, 5099: 1, 3610: 1, 6576: 1, 7389: 1}


#### Cumulative Probability Exact Match

In [36]:
df['CumProbaExactMatch'] = 0
df.sort_values(by=['DrawDate', 'PrizeType'], ascending=True, inplace=True)
df[df.LuckyNo.isin(get_permutations(2646))].tail(10)

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,1st_digit,2st_digit,3st_digit,4st_digit,DrawYear,DrawMonth,DrawWeek,DrawDay,DrawDayofweek,DrawDayofyear,CumProbaExactMatch
100843,479218,2018-06-26,ConsolationNo8,6264,314,0,108,8,6,2,6,4,2018,6,26,26,1,177,0
100968,479718,2018-07-04,SpecialNo8,6642,123,172,8,77,6,6,4,2,2018,7,27,4,2,185,0
101812,483418,2018-09-19,SpecialNo10,6246,1869,158,77,95,6,2,4,6,2018,9,38,19,2,262,0
102879,488118,2018-12-23,1stPrizeNo,6642,172,0,95,27,6,6,4,2,2018,12,51,23,6,357,0
103165,489319,2019-01-19,ConsolationNo7,4662,1652,0,27,36,4,6,6,2,2019,1,3,19,5,19,0
103599,491219,2019-02-24,ConsolationNo4,6246,158,66,36,41,6,2,4,6,2019,2,8,24,6,55,0
104010,493019,2019-04-06,ConsolationNo10,6426,832,0,41,25,6,4,2,6,2019,4,14,6,5,96,0
104268,494119,2019-05-01,ConsolationNo6,6246,66,0,25,31,6,2,4,6,2019,5,18,1,2,121,0
104570,495419,2019-06-01,ConsolationNo9,4626,1424,0,31,1,4,6,2,6,2019,6,22,1,5,152,0
104581,495519,2019-06-02,1stPrizeNo,2646,449,0,1,0,2,6,4,6,2019,6,22,2,6,153,0


In [37]:
df[df.LuckyNo.isin([2646])].tail(10)

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,1st_digit,2st_digit,3st_digit,4st_digit,DrawYear,DrawMonth,DrawWeek,DrawDay,DrawDayofweek,DrawDayofyear,CumProbaExactMatch
25544,151799,1999-05-26,SpecialNo10,2646,781,20,29,20,2,6,4,6,1999,5,21,26,2,146,0
25756,152699,1999-06-15,SpecialNo6,2646,20,127,20,4,2,6,4,6,1999,6,24,15,1,166,0
27086,158499,1999-10-20,SpecialNo2,2646,127,213,123,67,2,6,4,6,1999,10,42,20,2,293,0
29275,167900,2000-05-20,SpecialNo6,2646,213,346,13,7,2,6,4,6,2000,5,20,20,5,141,0
32817,183301,2001-05-01,SpecialNo6,2646,346,2324,0,0,2,6,4,6,2001,5,18,1,1,121,0
57213,289507,2007-09-11,ConsolationNo9,2646,2324,2303,38,99,2,6,4,6,2007,9,37,11,1,254,0
82438,399213,2013-12-31,ConsolationNo3,2646,2303,446,97,40,2,6,4,6,2013,12,1,31,1,365,0
87507,421215,2015-03-22,SpecialNo2,2646,446,1084,95,108,2,6,4,6,2015,3,12,22,6,81,0
99714,474318,2018-03-10,ConsolationNo6,2646,1084,449,7,108,2,6,4,6,2018,3,10,10,5,69,0
104581,495519,2019-06-02,1stPrizeNo,2646,449,0,1,0,2,6,4,6,2019,6,22,2,6,153,0


In [38]:
def get_cum_proba_match(df, numbers, field):
    """Cumulative probability for the numbers"""
    count = 0
    for index, row in df[df.LuckyNo.isin(numbers)].iterrows():
        count = count + 1
        # print(index)
        all_numbers = len(df.iloc[0:index]) + 1
        proba = round((count / TOTAL_NUMBERS) * 100,5)
        # print(index, row['DrawDate'], all_numbers, proba)
        # print(row)
        df[field].iloc[index] = proba
        

In [39]:
for no in range(0, TOTAL_NUMBERS):
    if no % 1000 == 0:
        print('Processing %s of 10000' % str(no))
    get_cum_proba_match(df, [no], 'CumProbaExactMatch')

Processing 0 of 10000
Processing 1000 of 10000
Processing 2000 of 10000
Processing 3000 of 10000
Processing 4000 of 10000
Processing 5000 of 10000
Processing 6000 of 10000
Processing 7000 of 10000
Processing 8000 of 10000
Processing 9000 of 10000


In [40]:
df.tail(10)

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,1st_digit,2st_digit,3st_digit,4st_digit,DrawYear,DrawMonth,DrawWeek,DrawDay,DrawDayofweek,DrawDayofyear,CumProbaExactMatch
104663,495819,2019-06-09,SpecialNo1,9569,705,0,95,0,9,5,6,9,2019,6,23,9,6,160,0.12
104664,495819,2019-06-09,SpecialNo10,6223,735,0,25,0,6,2,2,3,2019,6,23,9,6,160,0.06
104665,495819,2019-06-09,SpecialNo2,791,613,0,28,0,0,7,9,1,2019,6,23,9,6,160,0.11
104666,495819,2019-06-09,SpecialNo3,1218,1628,0,147,0,1,2,1,8,2019,6,23,9,6,160,0.07
104667,495819,2019-06-09,SpecialNo4,3669,14,0,14,0,3,6,6,9,2019,6,23,9,6,160,0.12
104668,495819,2019-06-09,SpecialNo5,9517,603,0,7,0,9,5,1,7,2019,6,23,9,6,160,0.14
104669,495819,2019-06-09,SpecialNo6,1844,1187,0,15,0,1,8,4,4,2019,6,23,9,6,160,0.11
104670,495819,2019-06-09,SpecialNo7,5050,120,0,120,0,5,0,5,0,2019,6,23,9,6,160,0.07
104671,495819,2019-06-09,SpecialNo8,9073,592,0,0,0,9,0,7,3,2019,6,23,9,6,160,0.14
104672,495819,2019-06-09,SpecialNo9,2976,2066,0,39,0,2,9,7,6,2019,6,23,9,6,160,0.08


#### Cumulative Probability Any Match

In [41]:
df['CumProbaAnyMatch'] = 0
df.sort_values(by=['DrawDate', 'PrizeType'], ascending=True, inplace=True)
df[df.LuckyNo.isin(get_permutations(2646))].tail(10)

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,1st_digit,2st_digit,3st_digit,4st_digit,DrawYear,DrawMonth,DrawWeek,DrawDay,DrawDayofweek,DrawDayofyear,CumProbaExactMatch,CumProbaAnyMatch
100843,479218,2018-06-26,ConsolationNo8,6264,314,0,108,8,6,2,6,4,2018,6,26,26,1,177,0.10,0
100968,479718,2018-07-04,SpecialNo8,6642,123,172,8,77,6,6,4,2,2018,7,27,4,2,185,0.12,0
101812,483418,2018-09-19,SpecialNo10,6246,1869,158,77,95,6,2,4,6,2018,9,38,19,2,262,0.09,0
102879,488118,2018-12-23,1stPrizeNo,6642,172,0,95,27,6,6,4,2,2018,12,51,23,6,357,0.13,0
103165,489319,2019-01-19,ConsolationNo7,4662,1652,0,27,36,4,6,6,2,2019,1,3,19,5,19,0.07,0
103599,491219,2019-02-24,ConsolationNo4,6246,158,66,36,41,6,2,4,6,2019,2,8,24,6,55,0.10,0
104010,493019,2019-04-06,ConsolationNo10,6426,832,0,41,25,6,4,2,6,2019,4,14,6,5,96,0.09,0
104268,494119,2019-05-01,ConsolationNo6,6246,66,0,25,31,6,2,4,6,2019,5,18,1,2,121,0.11,0
104570,495419,2019-06-01,ConsolationNo9,4626,1424,0,31,1,4,6,2,6,2019,6,22,1,5,152,0.16,0
104581,495519,2019-06-02,1stPrizeNo,2646,449,0,1,0,2,6,4,6,2019,6,22,2,6,153,0.13,0


In [53]:
no_list = []
for no in range(0, TOTAL_NUMBERS):
    if no % 1000 == 0:
        print('Processing %s of 10000' % str(no))
    if no not in no_list:
        nos = get_permutations(no)
        # breakpoint()
        no_list.extend(nos)
        get_cum_proba_match(df, nos , 'CumProbaAnyMatch')

Processing 0 of 10000
Processing 1000 of 10000
Processing 2000 of 10000
Processing 3000 of 10000
Processing 4000 of 10000
Processing 5000 of 10000
Processing 6000 of 10000
Processing 7000 of 10000
Processing 8000 of 10000
Processing 9000 of 10000


In [54]:
df.tail(10)

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,1st_digit,2st_digit,3st_digit,4st_digit,DrawYear,DrawMonth,DrawWeek,DrawDay,DrawDayofweek,DrawDayofyear,CumProbaExactMatch,CumProbaAnyMatch
104663,495819,2019-06-09,SpecialNo1,9569,705,0,95,0,9,5,6,9,2019,6,23,9,6,160,0.12,1.31
104664,495819,2019-06-09,SpecialNo10,6223,735,0,25,0,6,2,2,3,2019,6,23,9,6,160,0.06,1.17
104665,495819,2019-06-09,SpecialNo2,791,613,0,28,0,0,7,9,1,2019,6,23,9,6,160,0.11,2.56
104666,495819,2019-06-09,SpecialNo3,1218,1628,0,147,0,1,2,1,8,2019,6,23,9,6,160,0.07,1.38
104667,495819,2019-06-09,SpecialNo4,3669,14,0,14,0,3,6,6,9,2019,6,23,9,6,160,0.12,1.33
104668,495819,2019-06-09,SpecialNo5,9517,603,0,7,0,9,5,1,7,2019,6,23,9,6,160,0.14,2.25
104669,495819,2019-06-09,SpecialNo6,1844,1187,0,15,0,1,8,4,4,2019,6,23,9,6,160,0.11,1.46
104670,495819,2019-06-09,SpecialNo7,5050,120,0,120,0,5,0,5,0,2019,6,23,9,6,160,0.07,0.49
104671,495819,2019-06-09,SpecialNo8,9073,592,0,0,0,9,0,7,3,2019,6,23,9,6,160,0.14,2.64
104672,495819,2019-06-09,SpecialNo9,2976,2066,0,39,0,2,9,7,6,2019,6,23,9,6,160,0.08,2.32


In [55]:
df[df.LuckyNo.isin(get_permutations(2976))].tail(10)

,DrawNo,DrawDate,PrizeType,LuckyNo,DaysSinceLastExactMatch,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DaysUntilNextAnyMatch,1st_digit,2st_digit,3st_digit,4st_digit,DrawYear,DrawMonth,DrawWeek,DrawDay,DrawDayofweek,DrawDayofyear,CumProbaExactMatch,CumProbaAnyMatch
98534,469217,2017-12-02,3rdPrizeNo,9672,214,0,25,39,9,6,7,2,2017,12,48,2,5,336,0.14,2.23
98977,471118,2018-01-10,ConsolationNo5,2679,532,0,39,17,2,6,7,9,2018,1,2,10,2,10,0.07,2.24
99165,471918,2018-01-27,ConsolationNo9,2697,361,0,17,84,2,6,9,7,2018,1,4,27,5,27,0.09,2.25
100158,476218,2018-04-21,SpecialNo3,7296,651,0,84,78,7,2,9,6,2018,4,16,21,5,111,0.11,2.26
101006,479918,2018-07-08,SpecialNo1,9627,991,297,78,52,9,6,2,7,2018,7,27,8,6,189,0.14,2.27
101579,482418,2018-08-29,ConsolationNo8,6729,1103,0,52,39,6,7,2,9,2018,8,35,29,2,241,0.12,2.28
102043,484418,2018-10-07,SpecialNo2,7926,3123,0,39,167,7,9,2,6,2018,10,40,7,6,280,0.12,2.29
103853,492319,2019-03-23,ConsolationNo5,6972,4235,0,167,39,6,9,7,2,2019,3,12,23,5,82,0.06,2.30
104279,494119,2019-05-01,SpecialNo7,9627,297,0,39,39,9,6,2,7,2019,5,18,1,2,121,0.15,2.31
104672,495819,2019-06-09,SpecialNo9,2976,2066,0,39,0,2,9,7,6,2019,6,23,9,6,160,0.08,2.32


In [63]:
df.dtypes

DrawNo                              int64
DrawDate                   datetime64[ns]
PrizeType                          object
LuckyNo                             int64
DaysSinceLastExactMatch             int64
DaysUntilNextExactMatch             int64
DaysSinceLastAnyMatch               int64
DaysUntilNextAnyMatch               int64
1st_digit                           int64
2st_digit                           int64
3st_digit                           int64
4st_digit                           int64
DrawYear                            int64
DrawMonth                           int64
DrawWeek                            int64
DrawDay                             int64
DrawDayofweek                       int64
DrawDayofyear                       int64
CumProbaExactMatch                float64
CumProbaAnyMatch                  float64
dtype: object

#### Digits Frequencies

##### Create the columns

In [152]:
columns = ['DrawDate', '1st_digit', '2nd_digit', '3rd_digit', '4th_digit' ]
new_columns = []
import itertools
digits = list(range(0,10))
pairings = list(itertools.combinations(digits,2))
for p in pairings:
    new_columns.append("_".join(str(i) for i in p))

new_columns.extend(["0_0", "1_1", "2_2", "3_3", "4_4", "5_5", "6_6", "7_7", "8_8", "9_9"])
df.columns

['0_1', '0_2', '0_3', '0_4', '0_5', '0_6', '0_7', '0_8', '0_9', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8', '1_9', '2_3', '2_4', '2_5', '2_6', '2_7', '2_8', '2_9', '3_4', '3_5', '3_6', '3_7', '3_8', '3_9', '4_5', '4_6', '4_7', '4_8', '4_9', '5_6', '5_7', '5_8', '5_9', '6_7', '6_8', '6_9', '7_8', '7_9', '8_9', '0_0', '1_1', '2_2', '3_3', '4_4', '5_5', '6_6', '7_7', '8_8', '9_9']


Index(['DrawNo', 'DrawDate', 'PrizeType', 'LuckyNo', 'DaysSinceLastExactMatch',
       'DaysUntilNextExactMatch', 'DaysSinceLastAnyMatch',
       'DaysUntilNextAnyMatch', '1st_digit', 'DrawYear', 'DrawMonth',
       'DrawWeek', 'DrawDay', 'DrawDayofweek', 'DrawDayofyear',
       'CumProbaExactMatch', 'CumProbaAnyMatch', '2nd_digit', '3rd_digit',
       '4th_digit', '0_1', '0_2', '0_3', '0_4', '0_5', '0_6', '0_7', '0_8',
       '0_9', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8', '1_9', '2_3',
       '2_4', '2_5', '2_6', '2_7', '2_8', '2_9', '3_4', '3_5', '3_6', '3_7',
       '3_8', '3_9', '4_5', '4_6', '4_7', '4_8', '4_9', '5_6', '5_7', '5_8',
       '5_9', '6_7', '6_8', '6_9', '7_8', '7_9', '8_9', '0_0', '1_1', '2_2',
       '3_3', '4_4', '5_5', '6_6', '7_7', '8_8', '9_9'],
      dtype='object')

##### Derive the pairings

In [191]:
def get_pairings(df, no):
    pairs = list(set(itertools.combinations(no, 2)))
    cols = []
    for p in pairs:
      cols.append("_".join(d for d in sorted(p)))
    return cols

for c in new_columns:
    df[c] = 0

stats = {}
count = 0
for draw_date in df.DrawDate.unique():
    if count % 1000 == 0:
        print(f"Working on {draw_date}")
    count = count + 1
    for index, row in df[df.DrawDate == draw_date].iterrows():
        pairings =  get_pairings(df, pad(row.LuckyNo))
        for p in pairings:
            if p in df.columns:
                # df.at[index, p] =  df.at[index, p] + 1
                if p in stats:
                    stats[p] = stats[p] + 1
                else:
                    stats[p] = 1
            else:
                print("combination does not exists")
    for k in stats:
        df[k][df.DrawDate == draw_date] = stats[k]

# for index, row in df[df.DrawDate == '1992-05-06 00:00:00'].iterrows():
#     pairings =  get_pairings(df, pad(row.LuckyNo))
#     for p in pairings:
#         if p in df.columns:
#             if p in stats:
#                 stats[p] = stats[p] + 1
#             else:
#                 stats[p] = 1
#         else:
#           print("combination does not exists")
# for k in stats:
#     df[k][df.DrawDate == '1992-05-06 00:00:00'] = stats[k]

# df[df.DrawDate == '1992-05-06 00:00:00'][["DrawDate","LuckyNo"] + new_columns]

Working on 1992-05-06T00:00:00.000000000
Working on 1998-09-23T00:00:00.000000000
Working on 2004-10-31T00:00:00.000000000
Working on 2010-09-18T00:00:00.000000000
Working on 2016-04-26T00:00:00.000000000


In [205]:
df[["DrawDate","LuckyNo"] + new_columns].head(66)

,DrawDate,LuckyNo,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9,1_2,1_3,1_4,1_5,1_6,1_7,1_8,1_9,2_3,2_4,2_5,2_6,2_7,2_8,...,4_5,4_6,4_7,4_8,4_9,5_6,5_7,5_8,5_9,6_7,6_8,6_9,7_8,7_9,8_9,0_0,1_1,2_2,3_3,4_4,5_5,6_6,7_7,8_8,9_9
0,1992-05-06,19,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,2,4,1,...,2,3,3,0,4,1,2,1,3,1,1,1,1,2,3,3,5,2,0,1,1,2,2,1,1
1,1992-05-06,1124,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,2,4,1,...,2,3,3,0,4,1,2,1,3,1,1,1,1,2,3,3,5,2,0,1,1,2,2,1,1
2,1992-05-06,592,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,2,4,1,...,2,3,3,0,4,1,2,1,3,1,1,1,1,2,3,3,5,2,0,1,1,2,2,1,1
3,1992-05-06,5311,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,2,4,1,...,2,3,3,0,4,1,2,1,3,1,1,1,1,2,3,3,5,2,0,1,1,2,2,1,1
4,1992-05-06,407,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,2,4,1,...,2,3,3,0,4,1,2,1,3,1,1,1,1,2,3,3,5,2,0,1,1,2,2,1,1
5,1992-05-06,1949,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,2,4,1,...,2,3,3,0,4,1,2,1,3,1,1,1,1,2,3,3,5,2,0,1,1,2,2,1,1
6,1992-05-06,1606,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,2,4,1,...,2,3,3,0,4,1,2,1,3,1,1,1,1,2,3,3,5,2,0,1,1,2,2,1,1
7,1992-05-06,3775,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,2,4,1,...,2,3,3,0,4,1,2,1,3,1,1,1,1,2,3,3,5,2,0,1,1,2,2,1,1
8,1992-05-06,6226,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,2,4,1,...,2,3,3,0,4,1,2,1,3,1,1,1,1,2,3,3,5,2,0,1,1,2,2,1,1
9,1992-05-06,1271,3,1,1,3,4,3,1,0,5,3,4,4,2,2,3,1,4,0,2,2,2,4,1,...,2,3,3,0,4,1,2,1,3,1,1,1,1,2,3,3,5,2,0,1,1,2,2,1,1


In [201]:
# Total 55 pairing combinations
len(new_columns)

55

#### Digits Pairings

In [220]:
columns = ['1st_digit', '2nd_digit', '3rd_digit', '4th_digit']

In [221]:
for index, row in df[df.DrawDate == '1992-05-06 00:00:00'].iterrows():
    pass

In [222]:
df[columns].head(23)

,1st_digit,2nd_digit,3rd_digit,4th_digit
0,0,0,1,9
1,1,1,2,4
2,0,5,9,2
3,5,3,1,1
4,0,4,0,7
5,1,9,4,9
6,1,6,0,6
7,3,7,7,5
8,6,2,2,6
9,1,2,7,1


In [238]:
def get_digit_count(df, fld, digit, draw_date)->int:
    """Get digit count for a particular position"""
    return len(df[ (df[fld]== digit) & (df.DrawDate == draw_date)])

In [250]:
def get_draw_digit_freq(df, draw_date):
    """Get digits freqs per draw"""
    first_freq = second_freq = third_freq = forth_freq = 0
    for digit in range(0, 10):
        print(digit, 
              get_digit_count(df, "1st_digit", digit, draw_date), 
              get_digit_count(df, "2nd_digit", digit, draw_date),
              get_digit_count(df, "3rd_digit", digit, draw_date),
              get_digit_count(df, "4th_digit", digit, draw_date)
             )
        
get_draw_digit_freq(df, '1992-05-06 00:00:00')

0 4 1 4 1
1 4 4 4 4
2 1 4 3 1
3 2 2 2 0
4 2 4 1 3
5 2 1 3 3
6 2 2 1 3
7 4 1 3 2
8 1 1 1 1
9 1 3 1 5


#### Averaging gaps between dates

In [44]:
#### Visualisation

#### Digits Rolling Mean

In [45]:
# df['1st_digit_rolling_mean'] = df['1st_digit'].rolling(window=23).mean()
# df['2st_digit_rolling_mean'] = df['2st_digit'].rolling(window=23).mean()
# df['3st_digit_rolling_mean'] = df['3st_digit'].rolling(window=23).mean()
# df['4st_digit_rolling_mean'] = df['4st_digit'].rolling(window=23).mean()
# df['total_digits_rolling_mean'] = (df['1st_digit_rolling_mean'] + df['2st_digit_rolling_mean'] + df['3st_digit_rolling_mean'] + df['4st_digit_rolling_mean']) / 4

In [46]:
# df.head(46)